<a href="https://colab.research.google.com/github/ahyan9114/CSE-6040/blob/main/week06_session01_NB02_notes_on_pandas_part0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Main topics covered during today's session:_

Previous NB:

1. **Notes on Pandas, Part 0 (from Topic 7), introducing Series and DataFrame objects**

This NB:

2. **Notes on Pandas, Part 0 (from Topic 7), Series and DataFrame manipulations**

We are simply splitting up this NB that is on Vocareum and adding some explanatory notes for this Skills OH session. The code content is the same.

# Supplemental notes on Pandas

The [**pandas** library](https://pandas.pydata.org/) is a Python module for representing what we call "tibbles" in Topic 7. Beyond what you see there, this notebook has additional notes to help you understand how to manipulate objects in Pandas. These notes adapt those found in the recommended text, [Python for Data Analysis (2nd ed.)](http://shop.oreilly.com/product/0636920050896.do), which is written by the creator of pandas, [Wes McKinney](http://wesmckinney.com/).

**Versions.** The state of pandas is a bit in-flux, so it's important to be flexible and accommodate differences in functionality that might vary by version. The following code shows you how to check what version of pandas you have.

In [1]:
import pandas as pd  # Standard idiom for loading pandas

print("=== pandas version: {} ===\n".format(pd.__version__))

import sys
print("=== Python version ===\n{}".format(sys.version))

=== pandas version: 1.5.3 ===

=== Python version ===
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


The main object that pandas implements is the `DataFrame`, which is essentially a 2-D table. It's an ideal target for holding the tibbles of Topic+Notebook 7, and its design derives in part from [data frame objects in the R language](https://www.rdocumentation.org/packages/base/versions/3.5.1/topics/data.frame).

In addition to `DataFrame`, another important component of pandas is the `Series`, which is essentially one column of a `DataFrame` object (and, therefore, corresponds to variables and responses in a tibble).

In [2]:
from pandas import DataFrame, Series

# `Series` objects

A pandas [`Series`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) object is a column-oriented object that we will use to store a variable of a tibble.

In [3]:
obj = Series([-1, 2, -3, 4, -5])
print(f"`obj` has type `{type(obj)}`:\n\n{obj}")

`obj` has type `<class 'pandas.core.series.Series'>`:

0   -1
1    2
2   -3
3    4
4   -5
dtype: int64


Observe the common **base type** (`dtype: int64`) and **index** (element numbers).

Regarding the base type, a `Series` differs from a Python `list` in that the types of its elements are assumed to be the same. Doing so allows many operations on a `Series` to be faster than their counterparts for `list` objects, as in this search example.

In [4]:
from random import randint
n_ints = 10000000
max_value = 5*n_ints

print(f"""
Creating random `list` and `Series` objects:
- Length: {n_ints} elements
- Range: [{-max_value}, {max_value}]
""")
a_list = [randint(-max_value, max_value) for _ in range(n_ints)]
a_series = Series(a_list)

print("==> Estimating time to search the `list`:")
t_list_search = %timeit -o randint(-max_value, max_value) in a_list

print("\n==> Estimating time to search the `Series`:")
t_series_search = %timeit -o a_series.isin([randint(-max_value, max_value)])

print(f"\n==> (`list` time) divided by `Series` time is roughly {t_list_search.average / t_series_search.average:.1f}x")


Creating random `list` and `Series` objects:
- Length: 10000000 elements
- Range: [-50000000, 50000000]

==> Estimating time to search the `list`:
172 ms ± 36.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

==> Estimating time to search the `Series`:
47.5 ms ± 614 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

==> (`list` time) divided by `Series` time is roughly 3.6x


**If you create a `Series` with "mixed types," the `dtype` will become the most generic Python type, `object`.** (A deeper understanding of what this fact means requires some knowledge of object-oriented programming, but that won't be necessary for our course.)

**Note that we will cover this in detail during our introduction to numpy in week's Skills OH sessions. While it is not necessary (in this course) for you to fully comprehend how Python objects are created and stored, it is helpful for you in understanding how we manipulate data in pandas and numpy.**

In [5]:
obj2 = Series([-1, '2', -3, '4', -5])
obj2

0    -1
1     2
2    -3
3     4
4    -5
dtype: object

If you want to query the base type, use:

In [6]:
print(obj.dtype)
print(obj2.dtype)

int64
object


Regarding the index, it provides a convenient way to reference individual elements of the `Series`.

By default, a `Series` has an index that is akin to `range()` in standard Python, and effectively numbers the entries from 0 to `n-1`, where `n` is the length of the `Series`. A `Series` object also becomes list-like in how you reference its elements.

In [7]:
print("obj.index: {}".format(obj.index))
print("range(0, 5): {}".format(range(0, 5)))

obj.index: RangeIndex(start=0, stop=5, step=1)
range(0, 5): range(0, 5)


In [8]:
print("==> obj[2]:\n{}\n".format(obj[2]))
print("==> obj[3]:\n{}\n".format(obj[3]))
print("==> obj[1:3]:\n{}\n".format(obj[1:4]))

==> obj[2]:
-3

==> obj[3]:
4

==> obj[1:3]:
1    2
2   -3
3    4
dtype: int64



You can also use more complex index objects, like lists of integers and conditional masks.

In [9]:
I = [0, 2, 3]
obj[I] # Also: obj[[0, 2, 3]]

0   -1
2   -3
3    4
dtype: int64

**The code in the cell below creates what is called a "conditional mask", or a "boolean mask".**

The concept of a mask is important to understand in data analysis, and you will have the opportunity through the rest of the course to exercise this paradigm.

In [10]:
I_pos = obj > 0
print(type(I_pos), I_pos)

<class 'pandas.core.series.Series'> 0    False
1     True
2    False
3     True
4    False
dtype: bool


The first line above creates the mask, assigning a True/False value for each row in the Series that has a value greater than 0.

We can then apply the mask in the code below, to only return those rows with a True value (or False, if we so desire).

In [11]:
print(obj[I_pos])

1    2
3    4
dtype: int64


It is important to understand how the mask functions. It essentially creates a True/False index on the series (also on dataframes) that can be used to return the appropriate rows.

However, the index can be a more general structure, which effectively turns a `Series` object into something that is "dictionary-like."

Let's look in detail at the code below and what it does.

In [12]:
obj3 = Series([      1,    -2,       3,     -4,        5,      -6],
              ['alice', 'bob', 'carol', 'dave', 'esther', 'frank'])
obj3

alice     1
bob      -2
carol     3
dave     -4
esther    5
frank    -6
dtype: int64

The documentation for Python Series is a good place to go, to understand the syntax above.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html

In the above code, we are creating a Series using two lists. From the documentation, we can see that the first list is the data, and the second list is the indexes. Had we not included this list, it would have been numerical, per the documentation:

**Will default to RangeIndex (0, 1, 2, …, n) if not provided.**

But what this exercise shows us is that the index NEED NOT be numerical, and we can access the Series items using the index, whatever it is, as in the cell below.

In [13]:
print("* obj3['bob']: {}\n".format(obj3['bob']))
print("* obj3['carol']: {}\n".format(obj3['carol']))

* obj3['bob']: -2

* obj3['carol']: 3



In fact, you can construct a `Series` from a dictionary directly:

In [14]:
peeps = {'alice': 1, 'carol': 3, 'esther': 5, 'bob': -2, 'dave': -4, 'frank': -6}
obj4 = Series(peeps)
print(obj4)

alice     1
carol     3
esther    5
bob      -2
dave     -4
frank    -6
dtype: int64


The below 4 code cells are examples of various masks, including integer, index value, and boolean masks.

In [15]:
mujeres = [0, 2, 4] # list of integer offsets
print("* las mujeres of `obj3` at offsets {}:\n{}\n".format(mujeres, obj3[mujeres]))

* las mujeres of `obj3` at offsets [0, 2, 4]:
alice     1
carol     3
esther    5
dtype: int64



In [16]:
hombres = ['bob', 'dave', 'frank'] # list of index values
print("* hombres, by their names, {}:\n{}".format(hombres, obj3[hombres]))

* hombres, by their names, ['bob', 'dave', 'frank']:
bob     -2
dave    -4
frank   -6
dtype: int64


In [17]:
I_neg = obj3 < 0  # boolean mask
print(I_neg)

alice     False
bob        True
carol     False
dave       True
esther    False
frank      True
dtype: bool


In [18]:
print(obj3[I_neg])

bob     -2
dave    -4
frank   -6
dtype: int64


Because of the dictionary-like naming of `Series` elements, you can use the Python `in` operator in the same way you would a dictionary.

> Note: In the timing experiment comparing `list` search and `Series` search, you may have noticed that the benchmark does not use `in`, but rather, `Series.isin`. Why is that?

In [19]:
print('carol' in peeps)  # from the dictionary above
print('carol' in obj3)  # in the Series object

True
True


Basic arithmetic works on `Series` as vector-like operations.

In [20]:
print(obj3, "\n")
print(obj3 + 5, "\n")
print(obj3 + 5 > 0, "\n")  # creates a boolean mask
print((-2.5 * obj3) + (obj3 + 5))

alice     1
bob      -2
carol     3
dave     -4
esther    5
frank    -6
dtype: int64 

alice      6
bob        3
carol      8
dave       1
esther    10
frank     -1
dtype: int64 

alice      True
bob        True
carol      True
dave       True
esther     True
frank     False
dtype: bool 

alice      3.5
bob        8.0
carol      0.5
dave      11.0
esther    -2.5
frank     14.0
dtype: float64


A `Series` object also supports vector-style operations with automatic alignment based on index values.

In [21]:
print(obj3)

alice     1
bob      -2
carol     3
dave     -4
esther    5
frank    -6
dtype: int64


In [22]:
# recall mujeres definition
# mujeres = [0, 2, 4] # list of integer offsets
obj_l = obj3[mujeres]
obj_l

alice     1
carol     3
esther    5
dtype: int64

In [23]:
# note that we are doing addition
obj3 + obj_l

alice      2.0
bob        NaN
carol      6.0
dave       NaN
esther    10.0
frank      NaN
dtype: float64

Observe what happened with undefined elements. If you are familiar with relational databases, this behavior is akin to an _outer-join_, which we will cover in our Skills OH SQL module next week.

**For our Skills OH session, we will stop here in our introduction of Series objects, as we will introduce the .apply() function in our next session.**

**We will go back now and introduce DataFrames.**

Another useful transformation is the `.apply(fun)` method. It returns a copy of the `Series` where the function `fun` has been applied to each element. For example:

In [24]:
abs(-5) # Python built-in function

5

In [25]:
obj3 # Recall

alice     1
bob      -2
carol     3
dave     -4
esther    5
frank    -6
dtype: int64

In [26]:
obj3.apply(abs)

alice     1
bob       2
carol     3
dave      4
esther    5
frank     6
dtype: int64

In [27]:
obj3 # Note: `.apply()` returned a copy, so the original is untouched

alice     1
bob      -2
carol     3
dave     -4
esther    5
frank    -6
dtype: int64

A `Series` may be _named_, too.

In [28]:
print(obj3.name)

None


In [29]:
obj3.name = 'peep'
obj3

alice     1
bob      -2
carol     3
dave     -4
esther    5
frank    -6
Name: peep, dtype: int64

When we move on to `DataFrame` objects, you'll see why names matter.

# `DataFrame` objects

A pandas [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) object is a table whose columns are `Series` objects, all keyed on the same index. It's the perfect container for what we have been referring to as a tibble.

In [30]:
cafes = DataFrame({'name': ['east pole', 'chrome yellow', 'brash', 'taproom', '3heart', 'spiller park pcm', 'refuge', 'toptime'],
                   'zip': [30324, 30312, 30318, 30317, 30306, 30308, 30303, 30318],
                   'poc': ['jared', 'kelly', 'matt', 'jonathan', 'nhan', 'dale', 'kitti', 'nolan']})
print("type:", type(cafes))
print(cafes)

type: <class 'pandas.core.frame.DataFrame'>
               name    zip       poc
0         east pole  30324     jared
1     chrome yellow  30312     kelly
2             brash  30318      matt
3           taproom  30317  jonathan
4            3heart  30306      nhan
5  spiller park pcm  30308      dale
6            refuge  30303     kitti
7           toptime  30318     nolan


**Let's take a look in detail at the syntax above.**

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

Similar to the Series syntax, the first two parameters are data and index, and it also adds a third parameter, which is the column names.

In the above code, note that we are only passing in the "data" parameter, as we are passing in a dictionary (note the enclosing brackets) with keys and values. The keys become the column names and the values become the dataframe rows. Because we did not pass in anything for the index, it defaults to the numeric range, same as the Series.

In [31]:
display(cafes) # Or just `cafes` as the last line of a cell

,name,zip,poc
0,east pole,30324,jared
1,chrome yellow,30312,kelly
2,brash,30318,matt
3,taproom,30317,jonathan
4,3heart,30306,nhan
5,spiller park pcm,30308,dale
6,refuge,30303,kitti
7,toptime,30318,nolan


The `DataFrame` has named columns, which are stored as an `Index` (more later!):

In [32]:
cafes.columns

Index(['name', 'zip', 'poc'], dtype='object')

Each column is a named `Series`:

(Go back to Slide 5 for a visual of this.)

In [33]:
type(cafes['zip']) # Aha!

pandas.core.series.Series

As you might expect, these `Series` objects should all have the same index.

In [34]:
cafes.index

RangeIndex(start=0, stop=8, step=1)

In [35]:
cafes.index == cafes['zip'].index

array([ True,  True,  True,  True,  True,  True,  True,  True])

In [36]:
cafes['zip'].index == cafes['poc'].index

array([ True,  True,  True,  True,  True,  True,  True,  True])

You may use complex indexing of columns.

The below code creates a column mask, similar to the masks introduced for series above, which were row masks.

In [37]:
target_fields = ['zip', 'poc']
cafes[target_fields]

,zip,poc
0,30324,jared
1,30312,kelly
2,30318,matt
3,30317,jonathan
4,30306,nhan
5,30308,dale
6,30303,kitti
7,30318,nolan


But slices apply to rows. Let's go back to our Slide on DataFrame Slicing for who this works.

In [38]:
cafes[1::2]

,name,zip,poc
1,chrome yellow,30312,kelly
3,taproom,30317,jonathan
5,spiller park pcm,30308,dale
7,toptime,30318,nolan


The index above is, by default, an integer range.

In [39]:
cafes.index

RangeIndex(start=0, stop=8, step=1)

In the cell below, we do three things:

1. Create a new DF from the 'poc' and 'zip' columns of the cafes df.
2. Add an index to the df, from the 'name'column of the cafes df.
3. Set the index name to None.

In [44]:
cafes2 = cafes[['poc', 'zip']]
cafes2.index = cafes['name']
cafes2.index.name = None
cafes2

,poc,zip
east pole,jared,30324
chrome yellow,kelly,30312
brash,matt,30318
taproom,jonathan,30317
3heart,nhan,30306
spiller park pcm,dale,30308
refuge,kitti,30303
toptime,nolan,30318


**Let's go back to the Slide to understand .loc and .iloc.**

You can access subsets of rows using the `.loc` field and index values:

In [41]:
cafes2.loc[['chrome yellow', '3heart']]

,poc,zip
chrome yellow,kelly,30312
3heart,nhan,30306


Alternatively, you can use integer offsets via the `.iloc` field, which is 0-based.

In [42]:
cafes2.iloc[[1, 3]]

,poc,zip
chrome yellow,kelly,30312
taproom,jonathan,30317
